<a href="https://colab.research.google.com/github/luqmanr/web-scraping/blob/master/instagram-crawler/instagram_crawler_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!python3 -m pip install opencv-contrib-python>=4.1.0
!pip install stdiomask
!apt install chromium-chromedriver
!pip install selenium
!pip install cp
!sudo apt-get install python-yaml
!sudo yum install python-yaml
!apt-get update
!apt install python3-dev python3-pip
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [0]:
import getpass

login_id = input('Your IG Account Username: ')
password = getpass.getpass('Your IG Account Password: ')

Your IG Account Username: tyogotest1
Your IG Account Password: ··········


In [0]:
usernames_target = [
  "tyogoutomo"
]

In [0]:
import os, time, errno, pickle
import re
import json
import csv
import yaml
import sys
import cp
import selenium
from selenium import webdriver
from urllib.request import urlopen
from optparse import OptionParser
from selenium import webdriver  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

## Login to instagram
def login_instagram():
    try:
      ## Go to Instagram login page to log in
      driver.get("https://www.instagram.com/accounts/login/")
      time.sleep(2)
      ## Find username and password textbox using each html name attribute, then send the variables that contain both respective value that has been declared on the second cell
      driver.find_element_by_name("username").send_keys(login_id)
      driver.find_element_by_name("password").send_keys(password)
      time.sleep(1)
      ## Find login button using its class name and click it to log in
      driver.find_element_by_class_name("L3NKy").click()
      return login_id
    except:
      sys.stderr.write("the Xpath of this element (login_instagram) has changed \n")
      time.sleep(3)
      driver.close()
      sys.exit()

## Get the full profile of each username 
def userProfile():
  try:
    ## Find following button in username profile page using its xpath, and then get the value of the number of followings for that username
    followingButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[3]/a/span'
    following_button = driver.find_element_by_xpath(followingButtonXpath).text
    ## Print the number of total followings
    print("Total Following: " + following_button)
  except:
    following_button = " "
    print("Total Following: " + following_button)
  try:
    ## Find post button in username profile page using its xpath, and then get the value of the number of posts for that username
    postButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[1]/span/span'
    post_button = driver.find_element_by_xpath(postButtonXpath).text
    ## Print the number of total posts
    print("Total posts: " + post_button)
  except:
    post_button = " "
    print("Total posts: " + post_button)
  try:
    ## Find full name panel in username profile page using its xpath, and then get the value of the full name for that username
    fullNameXpath = '/html/body/div[1]/section/main/div/header/section/div[2]/h1'
    full_name = driver.find_element_by_xpath(fullNameXpath).text
    ## Print the fullname
    print("Full Name: " + full_name)
  except:
    full_name = " "
    print("Full Name: " + full_name)
  return full_name, post_button, following_button

## Get the value of followers from each username
def followerButton():
    try:
      ## Find follower button in username profile page using its xpath, and then get the value of the number of followers for that username
      follButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[2]/a/span'
      follower_button = driver.find_element_by_xpath(follButtonXpath)
      printfoll = follower_button.get_attribute('title')
    except KeyError:
      ## Alternate xpath, in case xpath that declared above doesn't work, because Instagram change its API constantly
      follButtonXpath = '/html/body/div[1]/section/main/div/ul/li[2]/span'
      follower_button = driver.find_element_by_xpath(follButtonXpath)
      print("followerButton Xpath alt 2")
      pass
    except:
      pass
      sys.stderr.write("the Xpath of this element (followerButton) has changed \n")
    ## Parse variable printfoll that contain the number of followers, in case the number of followers is more than 999 (because when it reach thousand or million ',' will appear to sepperate the numbers)
    follValue = printfoll.split(',')
    ## Put together the numbers of followers after parsed before
    try:
      follvalue0 = follValue[0]
      follvalue1 = follValue[1]
      follvalue2 = follValue[2]
      follvalue = str(follvalue0 + follvalue1 + follvalue2)
      print("Total followers: " + follvalue)
      follower_button.click()
      return follvalue
    except KeyError:
      follvalue0 = follValue[0]
      follvalue1 = follValue[1]
      follvalue = str(follvalue0 + follvalue1)
      print("Total followers: " + follvalue)
      follower_button.click()
      return follvalue
    except:
      follvalue0 = follValue[0]
      follvalue = str(follvalue0)
      follvalue = follvalue0
      print("Total followers: " + follvalue)
      follower_button.click()
      return follvalue

## Get the followers list
def follGet(indexOfFollowerToBeCrawl):
  ## In order to make the program stable, limit the maximal of followers to crawl for each username
  follmax = 300
  ## Loop to get the value of username in Instagram followers list
  while indexOfFollowerToBeCrawl <= follmax:
    ## Convert arguments into string
    str_indexOfFollowerToBeCrawl = str(indexOfFollowerToBeCrawl)
    ## Find xpath of username in Instagram followers list
    try:
      follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_indexOfFollowerToBeCrawl + ']/div/div[1]/div[2]/div[1]/a'
    except KeyError:
      follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_indexOfFollowerToBeCrawl + ']/div/div[2]/div[1]/div/div/a'
    except:
        pass
        sys.stderr.write("the Xpath of this element (follGet) has changed \n")
    finally:
      follower_name = driver.find_element_by_xpath(follXpath)
      ## Get username value using .text
      follvalue = (follower_name.text)
      time.sleep(0.2)
      return follvalue
    ## Make the function break after reaching the maximal limit that declared above
    if n > follmax:
      break
    else:
      pass

'''def ID():
    urlmax = 300  #maximal userID to take (to avoid account block by Instagram)
    while counter <= urlmax:
        driver.switch_to.window(driver.window_handles[1])
        driver.get("https://www.instagram.com/"+USERstr+"/?__a=1")
        time.sleep(0.3)
        userID=driver.find_element_by_xpath('/html/body/pre')
        user=userID.get_attribute('innerHTML')
        s=user
        y=yaml.load(s)
        l=(y.get("logging_page_id"))
        IGid=(l.split('_'))
        IGidVal = IGid[1]
        driver.switch_to.window(driver.window_handles[0])
        if counter >= urlmax:
            print("ID Limit exceded")
            break
        return IGidVal

def myID():
    driver.switch_to.window(driver.window_handles[1])
    driver.get("https://www.instagram.com/"+login_id+"/?__a=1")
    time.sleep(0.3)
    userID=driver.find_element_by_xpath('/html/body/pre')
    user=userID.get_attribute('innerHTML')
    s=user
    y=yaml.load(s)
    l=(y.get("logging_page_id"))
    IGid=(l.split('_'))
    myIGidVal = IGid[1]
    driver.switch_to.window(driver.window_handles[0])
    return myIGidVal

def bot():
  scrollBot = 0 
  driver.get('https://www.instagram.com/')
  time.sleep(5)
  fBody = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/section')
  for scroll in range(0,10):
    driver.execute_script("window.scrollTo(0, 1300)")
    time.sleep(5)
    # scrollBot += 1
    # if scroll == 10:
    #   print("Bot scrolling finish")
    #   break
    # else:
    #   pass'''

################################################################################
infinity = 0x40000
maxCrawl = infinity
check = False
timeout = False
follGetError = False
timeoutValue = 10
timeoutCounter = 0
crawlCounter = 0
counter = 0
scroll = 0
folllist = []
follidlist = []
filename = "crawled_username.csv"

WINDOW_SIZE = "1360,768"
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
os.chdir("/content/gdrive/My Drive/")

driver = webdriver.Chrome(
    executable_path="/usr/lib/chromium-browser/chromedriver",
    chrome_options=chrome_options
)

## Change the directory to My Drive in Google Drive
os.chdir("/content/gdrive/My Drive/")
## Make a new directory "instagram-crawling" inside My Drive
try:
    os.mkdir("./instagram-crawling")
except:
    pass
## Change the directory to "/content/gdrive/My Drive/instagram-crawling"
os.chdir("/content/gdrive/My Drive/instagram-crawling")
## Make a new directory "instagram-usernames" inside instagram-crawling
try:
    os.mkdir("./instagram-usernames")
except:
    pass
## Change the directory to "/content/gdrive/My Drive/instagram-crawling/instagram-usernames"
os.chdir("/content/gdrive/My Drive/instagram-crawling/instagram-usernames")

## Try to open "crawled_username.csv" in order to check whether it already exist or not
try:
  open('crawled_username.csv', 'r')
  pass
## If "crawled_username.csv" doesn't exist, create a new csv file: "crawled_username.csv"
except:
  with open('crawled_username.csv', 'w', newline='') as csvfile:
    wr = csv.writer(csvfile)

login_instagram() ## Run login_instagram()
time.sleep(5) ## 5sec time delay to simulate real human interaction when entering Instagram
usernames_total = len(usernames_target) ## Define the length of username_total list that contain list of username(s) to crawl, that already assigned at the third cell
print("total of usernames to crawl: " + str(usernames_total))
## If the list is empty --> print error and exit
if usernames_total == 0:
  sys.stderr.write("There is no username to crawl. Please enter at least 1 username to crawl! \n")
  driver.close()
  driver.quit()
  sys.exit()
else:
  pass

## Crawl usernames that present on username_total list
while crawlCounter < maxCrawl:
  try:
    ## Loop each row of usernames_target list, based on index that generated by crawlCounter
    user_target = usernames_target[crawlCounter]
  except:
    sys.stderr.write("There is no username to crawl. The list is empty. \n")
    driver.close()
    sys.exit()
  ## Read "crawled_username.csv" to check which usernames that has already crawled before (crawled file(s) must be located inside /content/gdrive/My Drive/instagram-crawling/instagram-usernames)
  with open(filename) as csvread:
    reader = csv.reader(csvread)
    for row in reader:
      if user_target in row:
        print("This username is already crawled")
        crawlCounter += 1
        check = True
      else:
        check = False
  if check == True:
    continue
  elif check == False:
    pass
  
  # driver.execute_script("window.open('');")
  # driver.switch_to.window(driver.window_handles[0])

  ## Go to the username profile page
  driver.get('https://www.instagram.com/' + user_target + '/')
  crawlCounter += 1
  time.sleep(5)

  # timeoutCounter += 1
  # if timeoutCounter == 3:
  #   timeoutCounter = 0
  #   timeout = True
  #   print("timeout engage")
  # else:
  #   timeout = False
  #   pass

  ## Run followerButton()
  try:
    follower_button=followerButton()
  ## If error, declare that "This account is privete or doesn't exist" and break, so it will go to the next username in the loop
  except:
    sys.stderr.write("This account is private or doesn't exist \n")
    continue
  
  ## Convert follower_button to integer
  follower_value = int(follower_button)
  ## Run userProfile()
  full_name, post_button, following_button = userProfile()

  # myIDstr = str(myID())
  # print(myIDstr)
  
  ## If account doesn't have any follower, print "This username has no follower" and break, so it will go to the next username in the loop
  if follower_value == 0:
    sys.stderr.write("This username has no follower \n")
    break
  else:
    pass
  time.sleep(3) ## 3sec time delay to simulate real human interaction after opening followers list by pressing follower button

  ## Scroll followers list
  fBody = driver.find_element_by_xpath("//div[@class='isgrP']")
  for scroll in range(0,100):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', fBody)
    time.sleep(0.2)
  print("scrolling finish")
  time.sleep(1)
  
  ## Loop to get the value from followers list
  for i in range(1, follower_value):
    ## Run follGet() and put "i" that contain looping number 1 until the total followers ("follower_value") as a statement
    try:
      USERstr = str(follGet(i))
      follGetError = False
    ## If there is an error when collecting usernames from followers list, set follGetError to True and break
    except:
      sys.stderr.write("error occured \n")
      follGetError = True
      break
    print(str(i)+USERstr)
    if USERstr == "None":
      break
    else:
      pass
    # try:
    #   IDstr = str(ID())
    #   follGetError = False
    # except:
    #   sys.stderr.write("error occured \n")
    #   follGetError = True
    #   break
    # print(IDstr)
    # if IDstr == "None":
    #     IDstr = "ID limit exceded"
    # else:
    #     pass
    # follidlist.append(IDstr)
    usernames_target.append(USERstr)
    folllist.append(USERstr)

  ## If follGet not having error while collecting usernames from followers list, print folllist and divide the first row of the list and the rest of it, so it will place properly in csv file later
  if follGetError == False:
    print(folllist)
    folllistRow1 = folllist[0]
    folllist = folllist[1:follower_value]
  ## If follGetError is True, it will go to the next username in the loop
  else:
    continue
  
  ## Append username that has just been crawled inside the "crawled_username.csv"
  with open('crawled_username.csv', 'a+', newline='') as csvwrite:
    wr = csv.writer(csvwrite)
    wr.writerow([user_target])

  ## Make a new csv file and place lists and other informations that already collected during this crawling session inside
  with open('followers(' + user_target + ').csv', 'w', newline='') as csvfile:
      wr = csv.writer(csvfile)
      wr.writerow(["Root"] + ["Username"] + ["Full Name"] + ["Total Posts"] + ["Total Followers"] + ["Total Following"] + ["Followers"])
      wr.writerow([login_id] + [user_target] + [full_name] + [post_button] + [follower_button] + [following_button] + [folllistRow1])
      for follower in folllist:
        wr.writerow([""]+[""]+[""]+[""]+[""]+[""]+[follower])
  
  ## Open another window to avoid error while crawling in the next session
  driver.execute_script("window.open('');")
  ## Close the other window(s) that opened in this session
  try:
    driver.switch_to.window(driver.window_handles[1])
    driver.execute_script("window.close('');")
  except:
    pass
  try:
    driver.switch_to.window(driver.window_handles[0])
    driver.execute_script("window.close('');")
  except:
    pass
  
  ## Clear folllist
  folllist=[]
  ## Clear username that has been crawled in this session from usernames_target
  usernames_target.remove(user_target)

  # time.sleep(timeoutValue)
  # if timeout == True:
  #   print("restart")
    # if __name__ == '__main__':
    #   main()
    #   os.execv(sys.executable, [sys.executable] + sys.argv)
  # else:
  #   time.sleep(3)
  # bot()

## Clear usernames_target, just in case there is something left in the list
usernames_target=[]
time.sleep(5)
## Close ChromeDriver client
driver.close()